### Run the following code to import any libraries or packages required to run the rest of the script.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Run the following chunk of code to generate an eccentricity vs. adhesion time plot.

The user will need to adjust the base_Directory variable, and pay close attention to the directory structure of the script.

In [ ]:
base_Directory = ""
seconds_Per_Frame = 1.2
number_Of_Bins = 20
x_Values = np.linspace(0,1001,1000)

dynamic_Data = pd.read_csv(base_Directory + "Analysis/Video_Data/Dynamics_Data.csv")

average_Eccentricities = dynamic_Data["Average Eccentricity"].reset_index(drop = True)
attached_Frames = (dynamic_Data["Frame Detached"] - dynamic_Data["Frame Attached"]) + 1
attached_Times = attached_Frames*seconds_Per_Frame
attached_Times = attached_Times.reset_index(drop = True)

scatter_Values_Normoxia = []
bin_Midpoints_Log = []
bin_Width = int(max(attached_Times) - min(attached_Times))/number_Of_Bins
bin_Width_Log = (np.log10(max(attached_Times)) - np.log10(min(attached_Times)))/number_Of_Bins
manual_Bins = np.arange(min(attached_Times) - 1,max(attached_Times) + 1,bin_Width)
manual_Bins_Log = np.arange(np.log10(min(attached_Times)),np.log10(max(attached_Times)) + bin_Width_Log,bin_Width_Log)
for i in range(len(manual_Bins_Log) - 1):
    bin_Midpoints_Log.append((manual_Bins_Log[i + 1] + manual_Bins_Log[i])/2)
    
temp_Binning = []
errors = []
scatter_Values_Log = []
for i in range(len(manual_Bins_Log) - 1):
    for j in range(len(attached_Times)):
        if i == len(manual_Bins_Log) - 2:
            if np.log10(attached_Times[j]) >= manual_Bins_Log[i] and np.log10(attached_Times[j]) <= manual_Bins_Log[i + 1]:
                temp_Binning.append(average_Eccentricities[j])
            continue
        if np.log10(attached_Times[j]) >= manual_Bins_Log[i] and np.log10(attached_Times[j]) < manual_Bins_Log[i + 1]:
            temp_Binning.append(average_Eccentricities[j])
    if temp_Binning == []:
        scatter_Values_Log.append(0)
        errors.append(0)
    else:
        scatter_Values_Log.append(np.mean(temp_Binning))
        errors.append(np.std(temp_Binning))
    temp_Binning = []
scatter_Values_Log = np.array(scatter_Values_Log)
scatter_Values_Log[scatter_Values_Log == 0] = np.nan

fit_Data = pd.DataFrame({'Time':bin_Midpoints_Log, 'Eccentricity':scatter_Values_Log})
fit_Data = fit_Data.copy().dropna()

X = fit_Data["Time"]
Y = fit_Data["Eccentricity"]
X = sm.add_constant(X)
ols = sm.OLS(Y, X)
ols_result = ols.fit()
ols_result.summary()

fig = plt.figure(figsize=(8, 8))


gs = fig.add_gridspec(2, 2,  width_ratios=(7, 2), height_ratios=(2, 7),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)

ax = fig.add_subplot(gs[1, 0])
ax.scatter(10**np.array(bin_Midpoints_Log),scatter_Values_Log,facecolor = "blue", edgecolor = "blue",alpha = 0.5)
ax.plot(ols_result.params[1]*np.log10(x_Values) + ols_result.params[0],color = "black")
ax.plot(ols_result.conf_int()[1][1]*np.log10(x_Values) + ols_result.conf_int()[1][0],color = "black", ls = "--")
ax.plot(ols_result.conf_int()[0][1]*np.log10(x_Values) + ols_result.conf_int()[0][0],color = "black", ls = "--")
ax.set_xscale("log")
ax.set_xlim(1,2000)
ax.set_ylim(0,1)
x_Values = np.linspace(1,2000,1001)
plt.errorbar(10**np.array(bin_Midpoints_Log),scatter_Values_Log, yerr = errors, fmt = "", ecolor = "black",elinewidth = .75, color = "none", alpha = 0.5)

number_Of_Bins_Eccentricity = 20
eccentricity_Spacing = (np.max(dynamic_Data["Average Eccentricity"]) - np.min(dynamic_Data["Average Eccentricity"]))/(number_Of_Bins_Eccentricity)
eccentricity_Bins = np.arange(np.min(dynamic_Data["Average Eccentricity"]),np.max(dynamic_Data["Average Eccentricity"]) + eccentricity_Spacing,eccentricity_Spacing)

ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
ax_histy.tick_params(axis="y", labelleft=False)

N, bins, patches = ax_histy.hist(average_Eccentricities, edgecolor = "black", color = "blue", bins = eccentricity_Bins,orientation = "horizontal",density = True)
for direction in ["top", "bottom", "left", "right"]:
    ax_histy.spines[direction].set_visible(False)
ax_histy.tick_params(
    axis= "x",          
    which='both',      
    bottom=False,      
    top=False,         
    labelbottom=False) 
ax_histy.tick_params(
    axis= "y",          
    which='both',      
    left=False,      
    right=False,         
    labelleft=False) 
ax.set_xlabel("Adhesion Time")
ax.set_ylabel("Eccentricity")

plt.savefig(base_Directory + "Analysis/Outputs/Eccentricity_Vs_Adhesion_Time.png")
plt.savefig(base_Directory + "Analysis/Outputs/Eccentricity_Vs_Adhesion_Time.svg")

print("Slope P-Value = " + str(ols_result.pvalues[1]))